# Lab 3 - Model Deployment using Azure Machine Learning service

In this lab you will deploy a trained model to containers using an Azure Container Instance and and Azure Kubernetes Service using the Azure Machine Learning SDK.

## Download the datasets

The following cell will download the dataset used by this lab. Click into the following cell and use `Shift + Enter` to execute it

In [1]:
import os

main_path = os.path.abspath(os.path.curdir)
print("Current working directory is ", main_path)
data_path = os.path.join(main_path, 'data')
os.listdir(data_path)

Current working directory is  C:\Users\sasever\Desktop\SelfLearning\AzureML\AML-service-labs-master\starter-artifacts\jupyter\azure-ml-labs\03-model-deployment


['UsedCars_Affordability.csv']

## Train a simple model

This lab builds upon the lessons learned in the previous lab, but is self contained so you can work thru this lab without having to run a previous lab.

Read thru the following cell. Use `Shift + Enter` to execute the cell. Take a moment to look at the data loaded into the Pandas Dataframe - it contains data about used cars such as the price (in dollars), age (in years), KM (kilometers driven) and other attributes like weather it is automatic transimission, the number of doors, and the weight. 

In the function `train_eval_register_model` observe how the trained model is saved to the ./outputs folder along with the scaler that will be needed to scale inputs used later when scoring. Observe that we use `Model.register` to upload all files in the ./outputs folder to Azure Machine Learning as the model files. These model files will be retrieved later when the model is deployed into a container and operationalized as a web service.

In [2]:
# Step 1 - Load training data and define model training function
################################################################
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import azureml
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.model import Model 
import pickle
import json

# Verify AML SDK Installed
# view version history at https://pypi.org/project/azureml-sdk/#history 
print("SDK Version:", azureml.core.VERSION)


# Load our training data set
pathToCsvFile = os.path.join(data_path, 'UsedCars_Affordability.csv')
df_affordability = pd.read_csv(pathToCsvFile, delimiter=',')
print(df_affordability)

full_X = df_affordability[["Age", "KM"]]
full_Y = df_affordability[["Affordable"]]

# Define a helper method that will train, score and register the classifier using different settings
def train_eval_register_model(ws, experiment_name, model_name, full_X, full_Y,training_set_percentage):

    # start a training run by defining an experiment
    myexperiment = Experiment(ws, experiment_name)
    run = myexperiment.start_logging()

    train_X, test_X, train_Y, test_Y = train_test_split(full_X, full_Y, train_size=training_set_percentage, random_state=42)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(train_X)
    clf = linear_model.LogisticRegression(C=1)
    clf.fit(X_scaled, train_Y)

    scaled_inputs = scaler.transform(test_X)
    predictions = clf.predict(scaled_inputs)
    score = accuracy_score(test_Y, predictions)

    print("With %0.2f percent of data, model accuracy reached %0.4f." % (training_set_percentage, score))

    # Log the training metrics to Azure Machine Learning service run history
    run.log("Training_Set_Percentage", training_set_percentage)
    run.log("Accuracy", score)

    # Serialize the model to a pickle file in the outputs folder
    output_model_path = 'outputs/' + model_name + '.pkl'
    pickle.dump(clf,open(output_model_path,'wb'))
    print('Exported model to ', output_model_path)

    # Serialize the scaler as a pickle file in the same folder as the model
    output_scaler_path = 'outputs/' + 'scaler' + '.pkl'
    pickle.dump(scaler,open(output_scaler_path,'wb'))
    print('Exported scaler to ', output_scaler_path)

    # notice for the model_path, we supply the name of the outputs folder without a trailing slash
    # this will ensure both the model and the scaler get uploaded.
    registered_model = Model.register(model_path='outputs', model_name=model_name, workspace=ws)

    print(registered_model.name, registered_model.id, registered_model.version, sep = '\t')

    run.complete()

    return (registered_model, clf, scaler, score, run)

SDK Version: 1.0.6
      Age     KM  Affordable
0      23  46986           0
1      23  72937           0
2      24  41711           0
3      26  48000           0
4      30  38500           0
5      32  61000           0
6      27  94612           0
7      30  75889           0
8      27  19700           0
9      23  71138           0
10     25  31461           0
11     22  43610           0
12     25  32189           0
13     31  23000           0
14     32  34131           0
15     28  18739           0
16     30  34000           0
17     24  21716           0
18     24  25563           0
19     30  64359           0
20     30  67660           0
21     29  43905           0
22     28  56349           0
23     28  32220           0
24     29  25813           0
25     25  28450           0
26     27  34545           0
27     29  41415           0
28     28  44142           0
29     30  11090           0
...   ...    ...         ...
1406   70  44850           1
1407   69  44826        

In the following cell, we retrieve or create the AML Workspace and then train one instance of the model that we will deploy. In this step, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments. Execute the following cell.

In [3]:
# Step 2 - Retrieve the AML Workspace and Train a model
#######################################################

# Create a new Workspace or retrieve the existing one
#Provide the Subscription ID of your existing Azure subscription
subscription_id ='757c4165-0823-49f7-9678-5a85fe5e13cc'

# Provide values for the Resource Group and Workspace that will be created
resource_group = 'MLworkspace2'
workspace_name = 'snml2'
workspace_region = 'westeurope'  # eastus, westcentralus, southeastasia, australiaeast, westeurope


# By using the exist_ok param, if the worskpace already exists we get a reference to the existing workspace instead of an error
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

print("Workspace Provisioning complete.")


# Create an experiment, log metrics and register the created model
experiment_name = "Experiment-03-30"
model_name = "usedcarsmodel"
training_set_percentage = 0.50
registered_model, model, scaler, score, run = train_eval_register_model(ws, experiment_name, model_name, full_X, full_Y, training_set_percentage)

Workspace Provisioning complete.


C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\sasever\AppData\Local\Continuum\anaconda3\envs\amlpy36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


With 0.50 percent of data, model accuracy reached 0.9109.
Exported model to  outputs/usedcarsmodel.pkl
Exported scaler to  outputs/scaler.pkl
Registering model usedcarsmodel
usedcarsmodel	usedcarsmodel:4	4


## Download a version of a model from Azure Machine Learning

Once a model is registered with Azure Machine Learning, we can download the model files to any client and use them for scoring. In the following cell, you download the model you just registered, load both the scaler and model files retrieved by deserializing them into objects and then use them to perform a single prediction. Execute the following cell.

In [4]:
# Step 3 - Download the registered model, re-load  the model and verify it still works
######################################################################################
# Download the model to a local directory
model_path = Model.get_model_path(model_name, _workspace=ws)
print('Model download to: ' + model_path)
age = 60
km = 40000

# Re-load the model
scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))
scaled_input = scaler.transform([[age, km]])
model2 = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))

# Use the loaded model to make a prediction
prediction = model2.predict(scaled_input)
print(prediction)
prediction_json = json.dumps(prediction.tolist())
print(prediction_json)

Model download to: azureml-models\usedcarsmodel\4\outputs
[1]
[1]


## Create the container image configuration

When you deploy a model as web service to either ACI or AKS, you are deploying a Docker container. The first steps towards deploying involve defining the contents of that container. In the following cell, you create Conda Dependencies YAML file that describes what Python packages need to be installed in the container- in this case you specify scikit-learn, numpy and pandas. Execute the following cell.

In [5]:
# Step 4 - Create a Conda dependencies environment file
#######################################################
from azureml.core.conda_dependencies import CondaDependencies 

mycondaenv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'])

with open("mydeployenv.yml","w") as f:
    f.write(mycondaenv.serialize_to_string())

With Azure Machine Learning, you have full control over the logic of the webservice which includes how it loads your model, transforms web service inputs, uses the model for scoring and returns the result. The following cell will create a scoring web service and save the logic to the file score.py. Read thru the code that defines the webservice. Execute the following cell. The file will be deployed in the contents of the container image you are about to create in the upcoming steps.

In [6]:
#%%writefile score.py
scoring_service = """
import json
import os
import numpy as np
import pandas as pd
from sklearn import linear_model 
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from azureml.core import Run
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import pickle
from sklearn.externals import joblib

def init():
    try:
        # One-time initialization of predictive model and scaler
        from azureml.core.model import Model
        
        global trainedModel   
        global scaler

        model_name = "usedcarsmodel" 
        model_path = Model.get_model_path(model_name)
        print('Looking for models in: ', model_path)

        trainedModel = pickle.load(open(os.path.join(model_path,'usedcarsmodel.pkl'), 'rb'))
        
        scaler = pickle.load(open(os.path.join(model_path,'scaler.pkl'),'rb'))

    except Exception as e:
        print('Exception during init: ', str(e))

def run(input_json):     
    try:
        inputs = json.loads(input_json)

        #Scale the input
        scaled_input = scaler.transform(inputs)
        
        #Get the scored result
        prediction = json.dumps(trainedModel.predict(scaled_input).tolist())

    except Exception as e:
        prediction = str(e)
    return prediction
""" 

with open("score.py", "w") as file:
    file.write(scoring_service)

To create a Container Image, you need three things: the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file. Calling `ContainerImage.image_configuration` will capture all of the container image configuration in a single object. Execute the following cell.

In [7]:
# Step 5 - Create container image configuration
###############################################
# Build the ContainerImage
runtime = "python" 
driver_file = "score.py"
conda_file = "mydeployenv.yml"

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

## Deploy the container image to ACI

With the Container Image configuration in hand, you are almost ready to deploy to ACI. The next step is to define the size of the VM that ACI will use to run your Container. Execute the following cell to create this configuration.

In [8]:
# Step 6 - Create ACI configuration
####################################
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Azure ML ACI'}, 
    description = 'This is a great example.')

To deploy the container that operationalizes your model as a webservice, you can use `Webservice.deploy_from_model` which will use your registered model, and automate the creation of a new Container Image, and run the created container in ACI. Execute the following cell to deploy your webservice to ACI. This step will take **10-15 minutes** to complete.

In [9]:
# Step 7 -Deploy the webservice to ACI
######################################
service_name = "usedcarsmlservice01"

webservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name, 
  deployment_config=aci_config,
  models = [registered_model], 
  image_config=image_config, 
  )

webservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image usedcarsmlservice01:1, operation "Succeeded"
Creating service
Running..................
SucceededACI service creation operation finished, operation "Succeeded"


Once the webservice deployment completes, you can use the returned webservice object to invoke the webservice. Execute the following cell to invoke your webservice deployed to ACI.

In [10]:
# Step 8 - Test the ACI deployed webservice
###########################################
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = webservice.run(input_data=test_data)
print(result)


[1]


## Deploy the container image to AKS

Once you are familiar with the process for deploying a webservice to ACI, you will find the process for deploying to AKS to be similar with one additional step that creates the AKS cluster first. Execute the following cell to provision a small AKS cluster. This step will take about **15-20 minutes**.

In [11]:
# Step 9 - Provision an AKS cluster 
####################################
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

# Use the default configuration, overriding the default location to a known region that supports AKS
prov_config = AksCompute.provisioning_configuration(location='westus2')

aks_name = 'aks-cluster01' 

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

# Wait for cluster to be ready
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None


With your AKS cluster ready, now you can deploy your webservice. Once again, you need to provide a configuration for the size of resources allocated from the AKS cluster to run instances of your Container. Execute the following cell to deploy your webservice. This step will take **5-7 minutes**.

In [33]:
# Step 10 - Deploy webservice to AKS
####################################
# Create the web service configuration (using defaults)
aks_config = AksWebservice.deploy_configuration()

aks_service_name ='usedcarsaksservice'

aks_service = Webservice.deploy_from_model(
  workspace=ws, 
  name=aks_service_name, 
  deployment_config=aks_config,
  models = [registered_model], 
  image_config=image_config,
  deployment_target=aks_target
  )


aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating image
Image creation operation finished for image usedcarsaksservice:5, operation "Succeeded"
Creating service
Running...............................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy

As before, you can use the webservice object returned by the deploy_from_model method to invoke your deployed webservice. Execute the following cell to verify you can invoke the web service.

In [35]:
# Step 11 - Test the AKS deployed webservice
############################################
import json
age = 60
km = 40000
test_data  = json.dumps([[age,km]])
test_data
result = aks_service.run(input_data=test_data)
print(result)

[1]